In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import time

import sys
sys.path.append('./models')  # Make sure this is the correct path to the models

# Import the models from the converted Python files
import nbimporter
import import_ipynb

# Import model architectures
from attentionunet import AttentionUNet
from deeplabv3plus import DeepLabv3Plus
from denseunet import DenseUNet
from mfpunet import MFPUNet
from mobilenetunet import MobileNetUNet
from resnetunet import ResNetUNet
from unet import UNet

# Specify the number of input and output channels
n_channels = 3  # For RGB images
n_classes = 1   # For binary segmentation

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import cv2
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
import time

# Custom dataset for segmentation with robust transform handling
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, cache_data=False):
        """
        Dataset for image segmentation with optional caching and robust transform handling
        """
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.cache_data = cache_data
        self.cache = {}
        
        # List all images
        self.images = [img for img in os.listdir(image_dir) if img.endswith('.png') or img.endswith('.jpg')]
        
        # Validate all image-mask pairs exist
        self.valid_images = []
        for image_name in self.images:
            base_name = os.path.splitext(image_name)[0]
            # Look for mask with _annotation suffix
            mask_exists = False
            for ext in ['.png', '.jpg', '.jpeg', '.tif', '.bmp']:
                mask_path = os.path.join(mask_dir, base_name + "_annotation" + ext)
                if os.path.exists(mask_path):
                    mask_exists = True
                    break
            
            if mask_exists:
                self.valid_images.append(image_name)
        
        print(f"Found {len(self.valid_images)} valid image-mask pairs out of {len(self.images)} images")
    
    def __len__(self):
        return len(self.valid_images)
    
    def _robust_resize(self, image, mask, target_size=(256, 256)):
        """
        Resize image and mask robustly, ensuring they have the same dimensions
        """
        # Convert to numpy arrays if they're not already
        if isinstance(image, Image.Image):
            image = np.array(image)
        if isinstance(mask, Image.Image):
            mask = np.array(mask)
        
        # Resize using OpenCV to ensure consistent dimensions
        image_resized = cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)
        mask_resized = cv2.resize(mask, target_size, interpolation=cv2.INTER_NEAREST)
        
        return image_resized, mask_resized
    
    def __getitem__(self, idx):
        # Check if data is already cached
        if self.cache_data and idx in self.cache:
            return self.cache[idx]
            
        # Load image
        img_name = self.valid_images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        
        # Get base name without extension
        base_name = os.path.splitext(img_name)[0]
        
        # Find corresponding mask
        mask_path = None
        for ext in ['.png', '.jpg', '.jpeg', '.tif', '.bmp']:
            potential_path = os.path.join(self.mask_dir, base_name + "_annotation" + ext)
            if os.path.exists(potential_path):
                mask_path = potential_path
                break
        
        # Load mask
        mask = Image.open(mask_path).convert('L')  # Grayscale for mask
        
        # Robust resizing if needed
        image_arr, mask_arr = self._robust_resize(image, mask)
        
        # Apply transformations
        if self.transform:
            try:
                # For Albumentations transforms
                augmented = self.transform(image=image_arr, mask=mask_arr)
                image = augmented['image']
                mask = augmented['mask']
            except TypeError:
                # Fallback for simple transforms
                transform_result = self.transform(image=image_arr, mask=mask_arr)
                image = transform_result['image']
                mask = transform_result['mask']
        else:
            # Basic conversion if no transform
            image = torch.from_numpy(image_arr).permute(2, 0, 1).float() / 255.0
            mask = torch.from_numpy(mask_arr).unsqueeze(0).float() / 255.0
        
        # Convert mask to tensor and normalize
        mask_tensor = mask.unsqueeze(0).float() if len(mask.shape) == 2 else mask
        
        # Cache the result if caching is enabled
        if self.cache_data:
            self.cache[idx] = (image, mask_tensor, img_name)
            
        return image, mask_tensor, img_name

# The rest of your existing code remains the same

# Enhanced ModelEnsemble class with fixed individual model evaluation
class ModelEnsemble(nn.Module):
    def __init__(self, model_paths, device='cuda', dynamic_weights=False):
        """
        Initialize model ensemble with paths to .pth weight files
        
        Args:
            model_paths: Dictionary mapping model names to paths of .pth files
            device: Device to load models on ('cuda' or 'cpu')
            dynamic_weights: Whether to use dynamically calculated weights based on performance
        """
        super(ModelEnsemble, self).__init__()
        self.models = {}
        self.device = device
        self.dynamic_weights = dynamic_weights
        self.model_weights = {}  # For dynamic weighting
        
        if model_paths:
            self._load_models(model_paths)
    
    def _load_models(self, model_paths):
        # Load all models with proper initialization
        model_classes = {
            'attentionunet': AttentionUNet,
            'deeplabv3plus': DeepLabv3Plus,
            'denseunet': DenseUNet,
            'mfpunet': MFPUNet,
            'mobilenetunet': MobileNetUNet,
            'resnetunet': ResNetUNet,
            'unet': UNet
        }
     
        
        # Initial equal weights
        total_models = len(model_paths)
        for model_name in model_paths:
            self.model_weights[model_name] = 1.0 / total_models
        
        # Load each model
        for model_name, path in model_paths.items():
            if model_name in model_classes:
                try:
                    # Initialize model
                    model_class = model_classes[model_name]
                    model = model_class(n_channels=3, n_classes=1)
                    
                    # Load weights
                    checkpoint = torch.load(path, map_location=self.device)
                    if 'model_state_dict' in checkpoint:
                        model.load_state_dict(checkpoint['model_state_dict'])
                    else:
                        model.load_state_dict(checkpoint)
                    
                    # Move to device and set to eval mode
                    model.to(self.device)
                    model.eval()
                    
                    # Store model
                    self.models[model_name] = model
                    print(f"✅ Loaded {model_name} model from {path}")
                except Exception as e:
                    print(f"❌ Failed to load {model_name}: {e}")
            else:
                print(f"⚠️ Unknown model type: {model_name}")
        
        print(f"Loaded {len(self.models)} models for ensemble")
    
    def update_weights(self, validation_metrics):
        """
        Update model weights based on validation metrics
        
        Args:
            validation_metrics: Dictionary mapping model names to performance metrics
        """
        if not self.dynamic_weights:
            return
            
        # Calculate weights based on dice coefficient
        total_score = sum(validation_metrics.values())
        if total_score > 0:
            for model_name, score in validation_metrics.items():
                if model_name in self.model_weights:
                    self.model_weights[model_name] = score / total_score
        
        print("Updated model weights:")
        for model_name, weight in self.model_weights.items():
            print(f"  {model_name}: {weight:.4f}")

    def forward(self, x, method='average', return_individual=False):
        """
        Forward pass for ensemble models for binary segmentation
        
        Args:
            x: Input tensor
            method: Ensembling method ('average', 'weighted', 'max', 'vote', 'confidence')
            return_individual: Whether to return individual model predictions
            
        Returns:
            Ensemble prediction and optionally individual predictions
        """
        outputs = {}
        
        # Get predictions from all models
        with torch.no_grad():
            for name, model in self.models.items():
                output = model(x)
                # Ensure output has sigmoid applied for binary segmentation
                output = torch.sigmoid(output)
                outputs[name] = output
        
        # Stack outputs for processing
        stacked_outputs = torch.stack([outputs[name] for name in self.models.keys()], dim=0)
        
        if method == 'average':
            # Simple averaging of predictions
            result = torch.mean(stacked_outputs, dim=0)
        
        elif method == 'weighted':
            # Weighted average using model weights
            result = torch.zeros_like(list(outputs.values())[0])
            total_weight = 0
            
            for i, (name, output) in enumerate(outputs.items()):
                if name in self.model_weights:
                    result += output * self.model_weights[name]
                    total_weight += self.model_weights[name]
            
            # Normalize by sum of weights
            result = result / total_weight if total_weight > 0 else result
        
        elif method == 'max':
            # Take max confidence at each pixel
            result = torch.max(stacked_outputs, dim=0)[0]
        
        elif method == 'vote':
            # For binary segmentation, take majority vote (> 0.5) for each pixel
            votes = torch.zeros_like(list(outputs.values())[0])
            for output in outputs.values():
                votes += (output > 0.5).float()
            
            # Return 1 if majority of models vote for foreground, 0 otherwise
            result = (votes / len(outputs) > 0.5).float()
        
        elif method == 'confidence':
            # Weight each model's prediction by its confidence
            confidence_sum = torch.zeros_like(list(outputs.values())[0])
            weighted_sum = torch.zeros_like(list(outputs.values())[0])
            
            for output in outputs.values():
                # Calculate confidence as distance from 0.5
                confidence = torch.abs(output - 0.5) * 2  # Scales from 0 to 1
                confidence_sum += confidence
                weighted_sum += output * confidence
            
            # Normalize by total confidence
            result = weighted_sum / (confidence_sum + 1e-6)
            
        else:
            raise ValueError(f"Unknown ensemble method: {method}")
        
        if return_individual:
            return result, outputs
        else:
            return result

# Improved segmentation metrics calculation
def calculate_metrics(pred, target, smooth=1e-7):
    """
    Calculate multiple metrics for binary segmentation with robust handling
    
    Args:
        pred: Predicted binary mask (torch tensor)
        target: Ground truth binary mask (torch tensor)
        smooth: Small value to prevent division by zero
        
    Returns:
        Dictionary of metrics
    """
    # Ensure inputs are float tensors
    pred = pred.float()
    target = target.float()
    
    # Normalize and threshold predictions and targets
    pred_binary = (pred > 0.5).float()
    target_binary = (target > 0.5).float()
    
    # Flatten tensors
    pred_flat = pred_binary.view(-1)
    target_flat = target_binary.view(-1)
    
    # Compute confusion matrix components
    true_positives = torch.sum(pred_flat * target_flat)
    false_positives = torch.sum(pred_flat * (1 - target_flat))
    false_negatives = torch.sum((1 - pred_flat) * target_flat)
    true_negatives = torch.sum((1 - pred_flat) * (1 - target_flat))
    
    # Compute metrics
    # Dice Coefficient (F1 Score)
    dice = (2.0 * true_positives + smooth) / (2.0 * true_positives + false_positives + false_negatives + smooth)
    
    # Jaccard Index (IoU)
    iou = (true_positives + smooth) / (true_positives + false_positives + false_negatives + smooth)
    
    # Accuracy
    accuracy = (true_positives + true_negatives + smooth) / (true_positives + true_negatives + false_positives + false_negatives + smooth)
    
    # Precision
    precision = (true_positives + smooth) / (true_positives + false_positives + smooth)
    
    # Recall (Sensitivity)
    recall = (true_positives + smooth) / (true_positives + false_negatives + smooth)
    
    # Specificity
    specificity = (true_negatives + smooth) / (true_negatives + false_positives + smooth)
    
    # F1 Score
    f1 = (2 * precision * recall) / (precision + recall + smooth)
    
    return {
        'dice': dice.item(),
        'iou': iou.item(),
        'accuracy': accuracy.item(),
        'precision': precision.item(),
        'recall': recall.item(),
        'specificity': specificity.item(),
        'f1': f1.item()
    }

# Fixed evaluate_ensemble function with option to save all segmented images to a single folder
def evaluate_ensemble(ensemble, test_loader, device, ensemble_methods=None, threshold=0.5, 
                     output_dir='ensemble_results', save_all=True, validation_set=False,
                     single_output_folder=False):
    """
    Evaluate the ensemble model on test data using multiple methods
    
    Args:
        ensemble: Trained ensemble model
        test_loader: DataLoader for test data
        device: Device to evaluate on
        ensemble_methods: List of ensemble methods to evaluate
        threshold: Threshold for binary prediction
        output_dir: Directory to save prediction images
        save_all: Whether to save all prediction images
        validation_set: Whether this is evaluation on validation set (for updating weights)
        single_output_folder: Whether to save all segmented images in a single folder
        
    Returns:
        Dictionary of evaluation metrics for each method and visualization results
    """
    if ensemble_methods is None:
        ensemble_methods = ['average', 'weighted', 'max', 'vote', 'confidence']
    
    results = {method: {
        'dice': 0, 'iou': 0, 'accuracy': 0, 'precision': 0, 
        'recall': 0, 'specificity': 0, 'f1': 0
    } for method in ensemble_methods}
    
    # For tracking individual model performance
    individual_results = {model_name: 0 for model_name in ensemble.models}
    individual_batches = 0  # Counter for averaging individual results correctly
    
    num_batches = 0
    processing_times = {method: 0.0 for method in ensemble_methods}
    
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a single folder for all segmented images if requested
    if single_output_folder:
        all_segmentations_dir = os.path.join(output_dir, 'all_segmentations')
        os.makedirs(all_segmentations_dir, exist_ok=True)
    else:
        # Create method-specific folders as before
        for method in ensemble_methods:
            os.makedirs(os.path.join(output_dir, method), exist_ok=True)
    
    # For visualization
    vis_results = {method: {'images': [], 'masks': [], 'preds': [], 'filenames': []} for method in ensemble_methods}
    max_vis_samples = 4  # Store predictions for visualization
    
    # To collect all predictions and targets for ROC curve
    all_preds = {method: [] for method in ensemble_methods}
    all_targets = []
    
    with torch.no_grad():
        for batch_idx, (images, masks, filenames) in enumerate(tqdm(test_loader, desc="Evaluating")):
            images = images.to(device)
            masks = masks.to(device)
            
            # Store first batch images and masks for visualization
            if num_batches == 0:
                for i in range(min(max_vis_samples, images.size(0))):
                    for method in ensemble_methods:
                        vis_results[method]['images'].append(images[i].cpu())
                        vis_results[method]['masks'].append(masks[i].cpu())
                        vis_results[method]['filenames'].append(filenames[i])
            
            # For measuring individual model performance - evaluate on first 5 batches to get better average
            if batch_idx < 5:
                individual_batches += 1
                # Get predictions from each model independently
                with torch.no_grad():
                    for model_name, model in ensemble.models.items():
                        output = model(images)
                        output = torch.sigmoid(output)
                        pred = (output > threshold).float()
                        metrics = calculate_metrics(pred, masks)
                        individual_results[model_name] += metrics['dice']
            
            # Evaluate using each ensemble method
            for method in ensemble_methods:
                start_time = time.time()
                
                # Get ensemble predictions
                outputs = ensemble(images, method=method)
                
                # Apply threshold to get binary prediction
                preds = (outputs > threshold).float()
                
                # Calculate metrics
                metrics = calculate_metrics(preds, masks)
                for metric, value in metrics.items():
                    results[method][metric] += value
                
                # Store predictions for visualization (first batch only)
                if num_batches == 0:
                    for i in range(min(max_vis_samples, images.size(0))):
                        if i < len(vis_results[method]['images']):
                            vis_results[method]['preds'].append(preds[i].cpu())
                
                # Collect binary predictions for ROC curve
                all_preds[method].extend(outputs.cpu().numpy().flatten())
                if method == ensemble_methods[0]:  # Only need to collect targets once
                    all_targets.extend(masks.cpu().numpy().flatten())
                
                # Save all prediction images if requested
                if save_all:
                    for i in range(images.size(0)):
                        # Convert prediction to numpy array and scale to 0-255
                        pred_np = (preds[i].squeeze().cpu().numpy() * 255).astype(np.uint8)
                        # Create image from array
                        pred_img = Image.fromarray(pred_np)
                        # Get base name from filename
                        base_name = os.path.splitext(filenames[i])[0]
                        
                        if single_output_folder:
                            # Save in single folder with method name in filename
                            save_path = os.path.join(all_segmentations_dir, f"{base_name}_{method}_pred.png")
                        else:
                            # Save in method-specific folder as before
                            save_path = os.path.join(output_dir, method, f"{base_name}_pred.png")
                            
                        pred_img.save(save_path)
                
                # Measure processing time
                processing_times[method] += time.time() - start_time
            
            num_batches += 1
    
    # Average metrics over batches
    for method in ensemble_methods:
        for metric in results[method]:
            results[method][metric] /= num_batches
    
    # Average individual model results over the batches we evaluated them on
    for model_name in individual_results:
        if individual_batches > 0:  # Prevent division by zero
            individual_results[model_name] /= individual_batches
    
    # Calculate average processing time per batch
    for method in processing_times:
        processing_times[method] /= num_batches
    
    # If this is validation set, update ensemble weights
    if validation_set:
        ensemble.update_weights(individual_results)
    
    # Calculate ROC curve and AUC for each method
    roc_results = {}
    for method in ensemble_methods:
        try:
            # Convert predictions and targets to binary format
            binary_preds = (np.array(all_preds[method]) > threshold).astype(int)
            binary_targets = (np.array(all_targets) > threshold).astype(int)
            
            fpr, tpr, _ = roc_curve(binary_targets, binary_preds)
            roc_auc = auc(fpr, tpr)
            roc_results[method] = {'fpr': fpr, 'tpr': tpr, 'auc': roc_auc}
        except Exception as e:
            print(f"Warning: Could not calculate ROC curve for method {method}: {e}")
            # Provide default values so the function doesn't fail
            roc_results[method] = {'fpr': np.array([0, 1]), 'tpr': np.array([0, 1]), 'auc': 0.5}
    
    return results, vis_results, individual_results, processing_times, roc_results
    
# Enhanced visualization function
def visualize_ensemble_predictions(results, save_path=None, figsize=(20, 16)):
    """
    Visualize predictions from different ensemble methods
    
    Args:
        results: Dictionary with images, masks, and predictions for each method
        save_path: Path to save the visualization
        figsize: Size of the figure
    """
    methods = list(results.keys())
    num_methods = len(methods)
    num_samples = len(results[methods[0]]['images'])
    
    # Create a figure with rows for samples and columns for methods
    fig, axes = plt.subplots(num_samples, num_methods + 2, figsize=figsize)
    
    for i in range(num_samples):
        # Get the image and ground truth for this sample
        img = results[methods[0]]['images'][i].permute(1, 2, 0).numpy()
        mask = results[methods[0]]['masks'][i].squeeze().numpy()
        filename = results[methods[0]]['filenames'][i]
        
        # Normalize image for display
        img = (img - img.min()) / (img.max() - img.min())
        
        # Plot image
        if num_samples > 1:
            ax = axes[i, 0]
        else:
            ax = axes[0]
        ax.imshow(img)
        ax.set_title(f'Input: {filename}')
        ax.axis('off')
        
        # Plot ground truth
        if num_samples > 1:
            ax = axes[i, 1]
        else:
            ax = axes[1]
        ax.imshow(mask, cmap='gray')
        ax.set_title('Ground Truth')
        ax.axis('off')
        
        # Plot predictions for each method
        for j, method in enumerate(methods):
            pred = results[method]['preds'][i].squeeze().numpy()
            
            if num_samples > 1:
                ax = axes[i, j + 2]
            else:
                ax = axes[j + 2]
            ax.imshow(pred, cmap='gray')
            ax.set_title(f'{method.capitalize()}')
            ax.axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Prediction visualizations saved to {save_path}")
    
    plt.show()

# Function to visualize ROC curves
def visualize_roc_curves(roc_results, save_path=None):
    """
    Visualize ROC curves for different ensemble methods
    
    Args:
        roc_results: Dictionary with ROC curve data for each method
        save_path: Path to save the visualization
    """
    plt.figure(figsize=(10, 8))
    
    for method, data in roc_results.items():
        plt.plot(data['fpr'], data['tpr'], lw=2, label=f'{method} (AUC = {data["auc"]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curves')
    plt.legend(loc="lower right")
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"ROC curves saved to {save_path}")
    
    plt.show()

# Function to visualize confusion matrices
def visualize_confusion_matrices(results, test_loader, ensemble, device, threshold=0.5, save_path=None):
    """
    Visualize confusion matrices for different ensemble methods
    
    Args:
        results: Dictionary with evaluation results for each method
        test_loader: DataLoader for test data
        ensemble: Ensemble model
        device: Device to evaluate on
        threshold: Threshold for binary prediction
        save_path: Path to save the visualization
    """
    methods = list(results.keys())
    
    # Create figure for confusion matrices
    fig, axes = plt.subplots(1, len(methods), figsize=(5*len(methods), 5))
    if len(methods) == 1:
        axes = [axes]
    
    for method_idx, method in enumerate(methods):
        # Collect predictions and targets
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for images, masks, _ in test_loader:
                images = images.to(device)
                masks = masks.to(device)
                
                # Get predictions
                outputs = ensemble(images, method=method)
                preds = (outputs > threshold).float()
                
                # Collect binary predictions and targets
                all_preds.extend((preds > 0.5).int().cpu().numpy().flatten())
                all_targets.extend(masks.int().cpu().numpy().flatten())
        
        # Calculate confusion matrix
        cm = confusion_matrix(all_targets, all_preds)
        
        # Normalize confusion matrix
        #cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

        # After (with fix):
        row_sums = cm.sum(axis=1)[:, np.newaxis]
        # Replace any zero denominators with 1 to avoid division by zero
        row_sums[row_sums == 0] = 1
        cm_normalized = cm.astype('float') / row_sums
       
        # Plot confusion matrix
        sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues', 
                    xticklabels=['Background', 'Foreground'],
                    yticklabels=['Background', 'Foreground'],
                    ax=axes[method_idx])
        
        axes[method_idx].set_title(f'{method.capitalize()} Method')
        axes[method_idx].set_ylabel('True label')
        axes[method_idx].set_xlabel('Predicted label')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Confusion matrices saved to {save_path}")
    
    plt.show()

# Modified create_overlay_visualizations function to store in single folder
def create_overlay_visualizations(test_loader, ensemble, device, output_dir, ensemble_methods=None, threshold=0.5, single_output_folder=True):
    """
    Create overlay visualizations of predictions on input images
    
    Args:
        test_loader: DataLoader for test data
        ensemble: Ensemble model
        device: Device to evaluate on
        output_dir: Directory to save visualizations
        ensemble_methods: List of ensemble methods to evaluate
        threshold: Threshold for binary prediction
        single_output_folder: Whether to save all overlays in a single folder
    """
    if ensemble_methods is None:
        ensemble_methods = ['average', 'weighted', 'max', 'vote', 'confidence']
    
    # Create output directories
    if single_output_folder:
        # Create a single directory for all overlay visualizations
        overlays_dir = os.path.join(output_dir, 'all_overlays')
        os.makedirs(overlays_dir, exist_ok=True)
    else:
        # Create method-specific directories as before
        for method in ensemble_methods:
            os.makedirs(os.path.join(output_dir, f'{method}_overlay'), exist_ok=True)
    
    # Process a sample of images
    with torch.no_grad():
        for images, masks, filenames in tqdm(test_loader, desc="Creating overlays"):
            images = images.to(device)
            masks = masks.to(device)
            
            # Evaluate using each ensemble method
            for method in ensemble_methods:
                outputs = ensemble(images, method=method)
                preds = (outputs > threshold).float()
                
                # Create overlay for each image
                for i in range(images.size(0)):
                    # Convert original image to numpy and denormalize
                    img = images[i].cpu().permute(1, 2, 0).numpy()
                    # Assuming normalization with ImageNet stats
                    mean = np.array([0.485, 0.456, 0.406])
                    std = np.array([0.229, 0.224, 0.225])
                    img = std * img + mean
                    img = np.clip(img, 0, 1)
                    
                    # Convert to uint8 for overlay
                    img_uint8 = (img * 255).astype(np.uint8)
                    
                    # Get prediction and ground truth
                    pred = preds[i].squeeze().cpu().numpy()
                    mask = masks[i].squeeze().cpu().numpy()
                    
                    # Create overlay image
                    overlay = img_uint8.copy()
                    
                    # Add prediction (red) and ground truth (green) overlay
                    # True positive: yellow (red + green)
                    # False positive: red
                    # False negative: green
                    overlay[pred > 0.5, 0] = 0  # Remove blue where prediction is positive
                    overlay[pred > 0.5, 1] = 0  # Remove green where prediction is positive
                    overlay[pred > 0.5, 2] = 255  # Set red where prediction is positive
                    
                    overlay[mask > 0.5, 0] = 0  # Remove blue where ground truth is positive
                    overlay[mask > 0.5, 2] = 0  # Remove red where ground truth is positive
                    overlay[mask > 0.5, 1] = 255  # Set green where ground truth is positive
                    
                    # Set yellow for true positives (both pred and mask)
                    true_positive = (pred > 0.5) & (mask > 0.5)
                    overlay[true_positive, 1] = 255  # Green
                    overlay[true_positive, 2] = 255  # Red
                    
                    # Save overlay image
                    base_name = os.path.splitext(filenames[i])[0]
                    
                    if single_output_folder:
                        # Save in single folder with method name in filename
                        save_path = os.path.join(overlays_dir, f"{base_name}_{method}_overlay.png")
                    else:
                        # Save in method-specific folder as before
                        save_path = os.path.join(output_dir, f'{method}_overlay', f"{base_name}_overlay.png")
                    
                    cv2.imwrite(save_path, cv2.cvtColor(overlay, cv2.COLOR_RGB2BGR))
       

In [ ]:
# Modified main function to use single output folder for segmentations
def main():
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Path to your dataset
    image_dir = './data/images'
    mask_dir = './data/masks'
    
    # Output directory for all predictions
    output_dir = 'ensemble_results'
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a dedicated folder for all segmented images
    all_segmentations_dir = os.path.join(output_dir, 'all_segmentations')
    os.makedirs(all_segmentations_dir, exist_ok=True)
    
    # Try to import albumentations for transforms
    try:
        import albumentations as A
        from albumentations.pytorch import ToTensorV2
        
        transform = A.Compose([
            A.Resize(256, 256),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
        
        # Add some test-time augmentation for validation
        validation_transform = A.Compose([
            A.Resize(256, 256),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    except ImportError:
        print("Albumentations not found, using basic transforms")
        # Define a simple fallback transform
        class SimpleTransform:
            def __call__(self, image, mask):
                # Convert to tensor and normalize
                to_tensor = transforms.ToTensor()
                normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                
                return {
                    'image': normalize(to_tensor(image)),
                    'mask': torch.from_numpy(np.array(mask)).float()
                }
        
        transform = SimpleTransform()
        validation_transform = SimpleTransform()
    
    # Create dataset with optional caching
    test_dataset = SegmentationDataset(image_dir, mask_dir, transform=transform, cache_data=True)
    
    # Split dataset for validation and testing if needed
    test_size = len(test_dataset)
    validation_size = int(test_size * 0.2)  # 20% for validation
    
    # Use random_split to split the dataset
    if validation_size > 0:
        from torch.utils.data import random_split
        val_dataset, test_dataset = random_split(
            test_dataset, 
            [validation_size, test_size - validation_size],
            generator=torch.Generator().manual_seed(42)
        )
        print(f"Split dataset: {len(val_dataset)} validation, {len(test_dataset)} test samples")
    
    # Create data loaders
    batch_size = 4
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    if validation_size > 0:
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    # Path to your saved model weights
    model_paths = {
        'attentionunet': 'weights/best_AttentionUNet.pth',
        'deeplabv3plus': 'weights/best_deeplabv3plus.pth',
        'denseunet': 'weights/best_denseunet.pth',
        'mfpunet': 'weights/best_MFPUNet.pth',
        'mobilenetunet': 'weights/best_mobilenetunet.pth',
        'resnetunet': 'weights/best_ResNetUNet.pth',
        'unet': 'weights/best_UNet.pth'
    }
    
    # Check if model weights exist
    for model_name, path in model_paths.copy().items():
        if not os.path.exists(path):
            print(f"⚠️ Warning: Model weights not found at {path}")
            print(f"Removing {model_name} from ensemble")
            model_paths.pop(model_name)
    
    if not model_paths:
        print("❌ No model weights found. Please provide valid model weights.")
        return
    
    # Define ensemble methods to evaluate
    ensemble_methods = ['average', 'weighted', 'max', 'vote', 'confidence']
    
    # Create and initialize the ensemble
    print("Creating model ensemble...")
    ensemble = ModelEnsemble(model_paths, device, dynamic_weights=True)
    
    # First run validation to update weights if dynamic weighting is enabled
    if validation_size > 0 and ensemble.dynamic_weights:
        print("\nEvaluating on validation set to calibrate model weights...")
        val_results, _, individual_results, _, _ = evaluate_ensemble(
            ensemble, val_loader, device, 
            ensemble_methods=['average'], 
            output_dir=os.path.join(output_dir, 'validation'),
            save_all=False, 
            validation_set=True,
            single_output_folder=True  # Use a single folder for all segmentations
        )
        
        print("\nValidation Results (Average Method):")
        for metric, value in val_results['average'].items():
            print(f"  {metric}: {value:.4f}")
        
        print("\nIndividual Model Performance (Dice):")
        for model_name, dice in individual_results.items():
            print(f"  {model_name}: {dice:.4f}")
    
    # Evaluate on test set
    print("\nEvaluating on test set with all ensemble methods...")
    test_results, vis_results, individual_results, processing_times, roc_results = evaluate_ensemble(
        ensemble, test_loader, device, 
        ensemble_methods=ensemble_methods, 
        output_dir=output_dir,
        save_all=True,
        single_output_folder=True  # Use a single folder for all segmentations
    )
    
    # Print results
    print("\nTest Results:")
    for method in ensemble_methods:
        print(f"\n{method.capitalize()} Method:")
        for metric, value in test_results[method].items():
            print(f"  {metric}: {value:.4f}")
        print(f"  processing_time: {processing_times[method]:.4f} sec/batch")
    
    # Print individual model performance
    print("\nIndividual Model Performance (Dice):")
    for model_name, dice in individual_results.items():
        print(f"  {model_name}: {dice:.4f}")
    
    # Print model weights
    print("\nFinal Model Weights:")
    for model_name, weight in ensemble.model_weights.items():
        print(f"  {model_name}: {weight:.4f}")
    
    # Visualize predictions
    print("\nGenerating visualizations...")
    
    # Visualize predictions from different methods
    visualize_ensemble_predictions(
        vis_results, 
        save_path=os.path.join(output_dir, 'prediction_comparison.png')
    )
    
    # Visualize ROC curves
    visualize_roc_curves(
        roc_results,
        save_path=os.path.join(output_dir, 'roc_curves.png')
    )
    
    # Visualize confusion matrices
    visualize_confusion_matrices(
        test_results, test_loader, ensemble, device, 
        save_path=os.path.join(output_dir, 'confusion_matrices.png')
    )
    
    # Create overlay visualizations
    print("\nCreating overlay visualizations...")
    create_overlay_visualizations(
        test_loader, ensemble, device, output_dir,
        ensemble_methods=ensemble_methods
    )
    
    # Create a summary report
    summary_file = os.path.join(output_dir, 'summary_report.md')
    with open(summary_file, 'w') as f:
        f.write('# Medical Image Segmentation Ensemble Results\n\n')
        
        f.write('## Model Ensemble Configuration\n\n')
        f.write('Models included in the ensemble:\n\n')
        for model_name in ensemble.models:
            f.write(f'- {model_name}\n')
        
        f.write('\n## Model Weights\n\n')
        for model_name, weight in ensemble.model_weights.items():
            f.write(f'- {model_name}: {weight:.4f}\n')
        
        f.write('\n## Performance Metrics\n\n')
        for method in ensemble_methods:
            f.write(f'### {method.capitalize()} Method\n\n')
            for metric, value in test_results[method].items():
                f.write(f'- {metric}: {value:.4f}\n')
            f.write(f'- Processing time: {processing_times[method]:.4f} sec/batch\n')
            f.write(f'- ROC AUC: {roc_results[method]["auc"]:.4f}\n\n')
        
        f.write('\n## Visualization Results\n\n')
        f.write('The following visualizations were generated:\n\n')
        f.write('1. Prediction comparison across ensemble methods\n')
        f.write('2. ROC curves for each ensemble method\n')
        f.write('3. Confusion matrices for each ensemble method\n')
        f.write('4. Overlay visualizations showing predictions vs ground truth\n\n')
        f.write('5. All segmentation results are saved in the `all_segmentations` folder\n\n')
        
        f.write('## Conclusion\n\n')
        
        # Find best method based on dice score
        best_method = max(test_results, key=lambda x: test_results[x]['dice'])
        best_dice = test_results[best_method]['dice']
        
        f.write(f'The best performing ensemble method was **{best_method}** with a Dice score of **{best_dice:.4f}**.\n\n')
        
    print(f"\nSummary report saved to {summary_file}")
    print(f"All segmentation results saved to {os.path.join(output_dir, 'all_segmentations')}")
    print(f"All results saved to {output_dir}")

if __name__ == "__main__":
    main()